In [ ]:
from Get_data_DB import DataTransformer
from src.SCD1 import SCDType1Sync
from Connect_DB import mysql_conn, mysql_cursor, sql_server_conn, sql_server_cursor
import pandas as pd

# Tạo instance của class
transformer = DataTransformer()
mysql_query="select * from Lu_duc_test"
df1=transformer.fetch_from_mysql(mysql_query)

In [12]:
df1

,id,Ten
0,0,Lu Cuong
1,1,Nha
2,2,Mí Mì


In [4]:
sql_query="select * from Lu_duc_test"
df2=transformer.fetch_from_sql_server(sql_query)

In [5]:
df2

,id,Ten
0,0,Lu Cuong
1,1,Nha


In [6]:
syncer = SCDType1Sync(df1, df2, key_column='id',table_name="Lu_duc_test")
syncer.sync()